In [12]:
import pickle
import os.path
from whoosh.index import create_in
from whoosh.fields import *
from whoosh.qparser import QueryParser
from whoosh import qparser
import random
from nltk.corpus import stopwords
import nltk
import pyttsx3
import speech_recognition as sr
import glob
import pydub
import asr.align
from gtts import gTTS
import os
# nltk.download('stopwords')

In [2]:
transDict = pickle.load(open('transDict.p', 'rb'))
convertedDict = pickle.load(open('speechRecConvertDict.p', 'rb'))
queryList = pickle.load(open('queryList.p', 'rb'))
print(len(transDict))

2703


In [3]:
schema = Schema(audio=TEXT(stored=True), content=TEXT)
if not os.path.exists("index"):
    os.mkdir("index")
index = create_in("index", schema)
writer = index.writer()

In [4]:
if not os.path.exists("convertindex"):
    os.mkdir("convertindex")
convertindex = create_in("convertindex", schema)
convertwriter = convertindex.writer()

In [5]:
for a in transDict:
    writer.add_document(audio=u""+a, content=u""+transDict[a])
writer.commit()
for a in convertedDict:
    if(convertedDict[a]):
        convertwriter.add_document(audio=u""+a, content=u""+convertedDict[a])
convertwriter.commit()

In [6]:
searcher = index.searcher()
convertsearcher = convertindex.searcher()
query = QueryParser("content", index.schema, group=qparser.OrGroup).parse(u'antonia')
results = searcher.search(query)
convresults = convertsearcher.search(query)
for result in results:
    print(result)
    print(transDict[result['audio']])
    print(convertedDict[result['audio']])
print("-------------")
for result in convresults:
    print(result)
    print(convertedDict[result['audio']])

<Hit {'audio': '1988-147956-0023'}>
antonia pointed up to the sky and questioned me with her glance

Anthony appointed up to the sky and question to me with her glance
<Hit {'audio': '2035-147960-0011'}>
we decided that antonia should ride dude home and i would walk

we decided Antonia should ride dude home and I would walk
<Hit {'audio': '2035-147961-0013'}>
antonia's father uncovered one of his long bony legs and rubbed it rhythmically

Anthony's father uncovered one of his long bony legs and rubbed it with Nick Lee
<Hit {'audio': '1993-147964-0002'}>
i had wanted to get some picture books for yulka and antonia even yulka was able to read a little now

I had wanted to get some picture books for yoga and Antonia even yoga was able to read a little now
<Hit {'audio': '2035-147960-0002'}>
one day when i rode over to the shimerdas i found antonia starting off on foot for russian peter's house to borrow a spade ambrosch needed

one day when I rolled over to the shimerdas I found engineer 

KeyError: '6295-64301-0019'

In [7]:
origList = []
for a in transDict:
    origList.append(transDict[a])
lenOrig = len(origList)
stopWords = set(stopwords.words('english'))

In [41]:
queryList = []
for i in range(500):
    query = ""
    queryLen = 0
    while(queryLen<6):
        selectNum = random.randint(0, lenOrig)
#         print(selectNum)
        curr = origList[selectNum].rstrip().split(" ")
        selectNum = random.randint(0, len(curr)-1)
        if(curr[selectNum]!=" " and curr[selectNum] not in stopWords):
            query += curr[selectNum] + " "
            queryLen += 1
    queryList.append(query)
print(len(queryList))

500


In [42]:
for q in queryList:
    print(q)

yes missus hurriedly others mother friend 
looked wisely assured says white rancorous 
placed satisfied time attached would loss 
hair closed lip stage came blessed 
water see good outgrowth third regard 
across private struck decision rooms chief 
mister go dinner fakirs premature gymnast 
archway love wise see christian democratic 
strong daughter contemporaries appeals saw ran 
place either since patient looked one 
groom's threw promise tureen respects curses 
struggle arose scarcely child making august 
police thousand voice yes savages leisure 
seeing work said hepzibah rope muscles 
one honour must kept make young 
look particularly safely sensation variety they're 
enough fred perched typical mister sheep 
spread slight stumbled peter please see 
retreat george plebeian though banks son 
shaken every everybody anybody loiterers birdikins 
breaking pout movements thing oughtn't treasure 
week dome ascended sledges answer may 
gave men classes task feelings much 
mystery also qui

In [10]:
# engine = pyttsx3.init()
# engine.say('Sally sells seashells by the seashore.')
# engine.say('The quick brown fox jumped over the lazy dog.')
# engine.runAndWait()

In [27]:
from gtts import gTTS
import os
tts = gTTS(text='Good morning', lang='en')
tts.save("good.mp3")
os.system("mpg321 good.mp3")

32512

In [13]:
for i in range(len(queryList)):
    tts = gTTS(text=queryList[i], lang='en')
    tts.save("/datasets/home/35/335/cs253wgf/291/spokenkeywordsearch/GroundTruth/QueryAudio/"+str(i)+".mp3")
    os.system("mpg321 /datasets/home/35/335/cs253wgf/291/spokenkeywordsearch/GroundTruth/QueryAudio/"+str(i)+".mp3")

In [14]:
for filename in glob.iglob('/datasets/home/35/335/cs253wgf/291/spokenkeywordsearch/GroundTruth/**/*.mp3', recursive=True):
    try:
        sound = pydub.AudioSegment.from_mp3(filename)
    #     print(filename)
        vals = filename.rsplit(".", 1)
#         print(vals[0])
        sound.export(vals[0]+".wav", format="wav")
    except:
        print(filename)

In [16]:
r = sr.Recognizer()
def getTextFromAudio(filename):
    try:
        with sr.AudioFile(filename) as source:
            audio = r.record(source)
        audioText = ""
        audioText = r.recognize_google(audio)
        return audioText
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
 
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
        
    except:
        print("Couldn't convert")

In [17]:
count = 0
WER = 0
speechRecConvertDict = dict()
for filename in glob.iglob('/datasets/home/35/335/cs253wgf/291/spokenkeywordsearch/GroundTruth/**/*.wav', recursive=True):
#     print(filename)
    vals = filename.rsplit("/", 1)
    audioFileName = (vals[1].split(".", 1))[0]
#     print(audioFileName)
    text1 = queryList[int(audioFileName)]
    text2 = getTextFromAudio(filename)
    speechRecConvertDict[audioFileName] = text2
    if(not text1):
        text1 = ""
    if(not text2):
        text2 = ""
    WER = WER + asr.align.calculate_wer(text1, text2)
    count += 1
    if(count%10==0):
        print(WER)
        print(count)
print("Done")
print("Total count", count)
print("Average WER:", WER/count)

Google Speech Recognition could not understand audio
6.217302677663469
10
10.017778725989249
20
Google Speech Recognition could not understand audio
14.5854726587005
30
19.016069148711182
40
Google Speech Recognition could not understand audio
23.24303820643169
50
Google Speech Recognition could not understand audio
Google Speech Recognition could not understand audio
Google Speech Recognition could not understand audio
28.545223400932354
60
32.923068090155866
70
37.16874337929497
80
41.91983603247907
90
46.58292260624339
100
Google Speech Recognition could not understand audio
51.57837351379192
110
55.96580360347324
120
Google Speech Recognition could not understand audio
59.825002729024796
130
Google Speech Recognition could not understand audio
64.67445031529957
140
68.67205257029528
150
Google Speech Recognition could not understand audio
71.2995078204003
160
Google Speech Recognition could not understand audio
Google Speech Recognition could not understand audio
76.33907544925363


In [32]:
diffCount = 0
count = 0
for i in range(len(queryList)):
    if(queryList[i] and speechRecConvertDict[str(i)]):
        print(queryList[i])
        query1 = QueryParser("content", index.schema, group=qparser.OrGroup).parse(queryList[i])
        print(speechRecConvertDict[str(i)])
        query2 = QueryParser("content", index.schema, group=qparser.OrGroup).parse(speechRecConvertDict[str(i)])
        results = searcher.search(query1)
        convresults = convertsearcher.search(query2)
    #     result['audio']
        list1 = []
        list2 = []
        for res in results:
            list1.append(res['audio'])
        for res in convresults:
            list2.append(res['audio'])
        diffCount += len(set(list1).symmetric_difference(set(list2)))
        print(list1)
        print(list2)
        count += 1
        print(diffCount/count)

yes missus hurriedly others mother friend 
yes misses hurriedly others mother friend
['7976-105575-0019', '2277-149897-0007', '8297-275155-0005', '6345-93306-0009', '3081-166546-0073', '6313-66125-0012', '2428-83699-0001', '251-136532-0014', '3752-4944-0034', '7976-105575-0017']
['7976-105575-0019', '2277-149897-0007', '3081-166546-0073', '84-121123-0013', '6313-66125-0012', '2428-83699-0001', '251-136532-0014', '3752-4944-0034', '7976-105575-0017', '2035-147961-0027']
4.0
looked wisely assured says white rancorous 
looks wisely a shirt says white rancorous
['6295-244435-0015', '5694-64038-0006', '652-130737-0001', '3752-4943-0030', '3752-4944-0000', '5694-64038-0001', '3752-4944-0032', '2428-83705-0030', '3752-4943-0008', '1988-24833-0027']
['1993-147965-0000', '1988-24833-0023', '6295-244435-0015', '652-130737-0001', '3752-4944-0000', '5694-64038-0001', '2428-83705-0030', '5694-64038-0020', '5694-64038-0006', '3752-4943-0008']
5.0
placed satisfied time attached would loss 
play satis

['2035-147961-0024', '5895-34629-0001', '174-168635-0013', '2277-149897-0008', '2277-149896-0033', '251-118436-0014', '5536-43358-0002', '2277-149896-0034', '2428-83699-0030', '8297-275155-0020']
['2035-147961-0024', '2277-149897-0008', '2078-142845-0014', '7850-286674-0012', '2277-149896-0033', '7850-286674-0007', '2277-149896-0034', '2428-83699-0030', '8297-275155-0020', '6313-66125-0021']
10.80952380952381
gave men classes task feelings much 
feelings much
['3536-23268-0025', '1988-148538-0004', '3536-8226-0020', '1272-128104-0000', '7850-281318-0020', '2086-149220-0028', '3576-138058-0009', '2902-9008-0012', '6295-244435-0001', '5536-43363-0016']
['7850-281318-0020', '6295-244435-0001', '652-130737-0011', '5338-284437-0028', '174-84280-0011', '3853-163249-0056', '2412-153954-0006', '7850-73752-0007', '2035-152373-0003', '6241-61943-0005']
11.045454545454545
mystery also quick dear idea parted 
mystery also clicgear idea part is
['777-126732-0081', '1988-147956-0025', '8297-275155-0

cheapest gravel for word lit
['6345-93306-0012', '5694-64029-0018', '2035-147961-0016', '2035-147960-0004', '6241-61943-0010', '3536-8226-0017', '6295-244435-0030', '6345-93306-0002', '1993-147149-0001', '2803-154328-0000']
['6345-93306-0012', '2035-147960-0004', '6345-93306-0002', '2803-154328-0000', '1462-170142-0004', '8842-302196-0011', '2035-147960-0005', '2035-147961-0016', '3081-166546-0039', '5338-284437-0017']
11.159090909090908
motioning diocese staff embezzlement division burden 
motioning diocese staff embezzlement division burden
['2803-154328-0013', '7976-105575-0014', '1673-143397-0006', '3000-15664-0005', '3752-4943-0000', '7976-105575-0003', '84-121550-0000', '3853-163249-0037', '2428-83699-0017']
['7976-105575-0014', '1673-143397-0006', '3752-4943-0000', '3000-15664-0005', '7976-105575-0003', '84-121550-0000', '3853-163249-0037', '2428-83699-0017', '5338-24615-0002']
10.955555555555556
protect knew acquaintances mayonnaise circumstances best 
protect new acquaintances

['8297-275154-0005', '2803-154320-0014', '6313-66129-0032', '5895-34629-0025', '5895-34615-0005', '3752-4944-0048', '5895-34615-0018', '8297-275156-0009', '5895-34615-0014', '5895-34629-0014']
['5895-34615-0005', '8297-275154-0005', '5895-34629-0025', '5895-34622-0017', '777-126732-0052', '174-168635-0010', '5895-34622-0011', '2803-154328-0018']
11.078125
courts away horrible implies duty repulsed 
courts away horrible implies Duty
['2428-83699-0014', '6313-76958-0006', '2035-152373-0010', '6241-61943-0022', '5536-43363-0001', '7976-105575-0003', '5694-64029-0003', '3752-4944-0069', '1673-143397-0017', '7850-286674-0017']
['2428-83699-0014', '84-121123-0023', '6313-76958-0006', '2035-152373-0010', '6241-61943-0022', '5694-64029-0003', '3752-4944-0069', '7850-286674-0017', '1673-143397-0017', '2277-149896-0012']
10.96923076923077
mathematics owing explain future came would 
mathematics explain future
['2277-149896-0000', '2277-149897-0028', '1272-128104-0007', '7850-73752-0012', '2277-1

['3752-4944-0046', '652-130726-0022', '1993-147149-0015', '2035-147961-0031', '1988-24833-0026', '2035-147961-0016', '5895-34629-0023', '2035-147961-0001', '6313-66129-0004', '3536-8226-0029']
['3752-4944-0046', '2035-147961-0031', '6319-64726-0014', '2035-147961-0016', '1988-24833-0026', '2035-147961-0001', '6313-66129-0004', '3536-8226-0029', '3170-137482-0046', '3081-166546-0069']
10.421686746987952
manager sequins candle sliced offerings birdikins 
manager sequence candle spliced offerings
['7850-281318-0000', '652-130726-0006', '1919-142785-0033', '2428-83705-0029', '2277-149897-0009', '2277-149897-0020', '1462-170145-0016', '3170-137482-0046', '2428-83699-0040', '3081-166546-0062']
['3081-166546-0069', '652-130726-0006', '2428-83705-0029', '2277-149897-0009', '2277-149897-0020', '1462-170145-0016', '2428-83699-0040', '3081-166546-0062', '5536-43358-0007']
10.357142857142858
position homes kindred praise forests friend 
position homes Kindred praise Forest friend
['777-126732-0044

...

['7976-105575-0026', '1272-135031-0021', '2035-147961-0009', '3000-15664-0022', '7976-105575-0013', '7850-286674-0017', '2035-147960-0010', '8297-275156-0005', '2277-149896-0012', '2277-149897-0009']
['8842-304647-0005', '3000-15664-0022', '2078-142845-0025', '2035-147960-0010', '422-122949-0014']
10.830945558739256
fall appearance saw evening thus joe 
call appearance
['3752-4943-0020', '5895-34622-0010', '6313-66129-0006', '5338-284437-0000', '2428-83705-0006', '3536-23268-0004', '3536-23268-0015', '3081-166546-0039', '3536-23268-0009', '3000-15664-0027']
['777-126732-0018', '777-126732-0039', '5895-34622-0010', '5338-284437-0000', '3000-15664-0027', '3752-4943-0026', '3081-166546-0024', '6313-66125-0019', '6241-61946-0014', '5895-34615-0003']
10.84
granted remarkable affirmative away added last 
what is remarkable affirmative
['84-121123-0019', '5895-34629-0020', '5536-43363-0012', '2428-83705-0000', '1919-142785-0029', '1988-147956-0029', '8297-275155-0008', '777-126732-0038', '385

['174-50561-0005', '1988-24833-0021', '7976-110523-0006', '2277-149897-0027', '174-50561-0013', '8842-302201-0008', '777-126732-0048', '6313-66129-0029', '174-50561-0008', '1988-24833-0003']
['2277-149897-0027', '174-50561-0005', '8842-302196-0006', '1988-24833-0021', '7976-110523-0006', '2277-149897-0016', '2428-83699-0003', '8842-302201-0008', '3536-23268-0013', '777-126732-0048']
10.886486486486486
philly's elevator daguerreotype covered stairs nature 
Chili's elevator daguerreotype nature
['2086-149220-0027', '5895-34615-0001', '2428-83705-0031', '2277-149874-0007', '1272-141231-0001', '2086-149220-0022', '2086-149220-0049', '2035-147961-0011', '3081-166546-0085', '1462-170138-0011']
['2086-149220-0027', '5895-34615-0001', '2428-83705-0031', '2086-149220-0022', '3081-166546-0085', '84-121550-0004', '174-168635-0008', '5536-43358-0013', '2086-149214-0003', '6241-61946-0013']
10.884097035040432
perfection well bartley school salt ill 
Protection One
['2078-142845-0020', '1919-142785-

11.007692307692308
stupirti slight break simple form hundred 
super cheap light brake simple form hundred
['8842-304647-0001', '1462-170142-0034', '777-126732-0021', '777-126732-0029', '7976-110124-0024', '2086-149220-0024', '2035-147961-0008', '6319-64726-0006', '2277-149874-0005', '777-126732-0036']
['1919-142785-0022', '2086-149220-0024', '6319-64726-0006', '2035-147961-0008', '652-130737-0007', '5895-34629-0018', '6313-66125-0006', '2428-83705-0018', '6295-64301-0022', '1988-148538-0003']
11.0153452685422
dead time peter whispered perfectly clear 
Deadtime Peter whispered perfectly clear
['2035-147961-0001', '777-126732-0017', '2412-153947-0001', '1462-170145-0013', '1462-170142-0032', '2035-147961-0031', '1462-170142-0042', '2035-147961-0024', '2035-147961-0016', '6313-66125-0003']
['2035-147961-0001', '1462-170145-0013', '1462-170142-0032', '2412-153947-0001', '1462-170142-0042', '2035-147961-0031', '2035-147961-0016', '6313-66125-0003', '5694-64038-0013', '1462-170142-0025']
10.

['2428-83705-0008', '2412-153947-0016', '1272-141231-0021', '6313-66125-0018', '1993-147149-0028', '6313-76958-0016', '251-118436-0020', '2035-152373-0017', '652-130737-0011', '6345-64257-0013']
['2412-153947-0016', '1272-141231-0021', '1993-147149-0028', '2035-152373-0017', '652-130737-0011', '6345-64257-0013', '3081-166546-0055', '6319-64726-0006', '5338-284437-0002', '5895-34629-0018']
10.91219512195122
married food sleep thing unsupported allowed 
allowed
['3853-163249-0042', '7850-286674-0010', '3752-4944-0010', '2428-83699-0030', '2902-9008-0008', '3000-15664-0006', '1272-141231-0023', '6313-76958-0015', '6313-66125-0026', '6345-93302-0003']
['2078-142845-0035', '3536-8226-0031', '2428-83699-0030', '6345-93302-0003', '3853-163249-0042', '5895-34622-0013', '2412-153947-0006']
10.912408759124087
made house pinkies glad consisting everything 
glad consisting everything
['1988-147956-0005', '6241-61943-0021', '1462-170142-0018', '652-130726-0006', '1462-170138-0026', '251-137823-0000

['3853-163249-0002', '5338-284437-0014', '3752-4944-0008', '3752-4943-0022', '6313-66125-0024', '2803-154328-0010', '251-118436-0023', '3752-4943-0006', '1993-147966-0003', '3081-166546-0025']
['5338-284437-0014', '1462-170145-0011', '3752-4943-0022', '2803-154328-0010', '6319-57405-0005', '3081-166546-0025', '7976-105575-0013', '2035-147961-0025', '6313-66125-0010', '6295-244435-0028']
10.914351851851851
exposed good properties tom tardy large 
exposed good properties Tom tardy large
['652-130737-0005', '251-137823-0020', '3170-137482-0004', '251-137823-0024', '1988-24833-0027', '652-130726-0001', '8842-304647-0013', '1988-24833-0022', '1988-24833-0023', '3170-137482-0012']
['652-130737-0005', '251-137823-0020', '3170-137482-0004', '251-137823-0024', '1988-24833-0022', '652-130726-0001', '8842-304647-0013', '251-137823-0009', '251-137823-0013', '3170-137482-0012']
10.89838337182448
long approached ought pour north fellowship 
long a
['652-129742-0008', '3752-4944-0014', '3752-4943-000

In [33]:
print(diffCount)

4873


In [34]:
print(speechRecConvertDict)

{'289': "Macy's warmly Tom drainage misses", '341': None, '75': 'understand late instruments', '9': 'play Seether since patient looks like', '402': 'friends', '83': 'problem.', '235': 'proper and sound niggard Miss times', '223': 'take an interest events bed dresser 3', '424': 'adopted friend Rich served on cloud defeat', '109': 'followed beautiful Celestial Sunday telephone family', '158': 'selfishness suppressing care', '405': 'Loom stick in ground chamber', '180': 'anyway dear women missionaries climax', '295': 'talked seriousness Icelandic boys Sydney seat', '322': 'Lantern', '141': 'nourishment attempt', '439': 'mini arrangement', '377': 'Wildside still 20 sensation pickets', '381': 'Bridge Stoops arched say people take', '216': 'zingiber Bell there for kitchen Liberty', '413': 'would cat complications Time Lady first', '477': 'Milnor credit generalship play songs', '263': None, '137': None, '246': 'contracted hands in Denver', '458': None, '414': "Chili's elevator daguerreotype n